In [2]:
%tensorflow_version 1.x
import os 
os.chdir("/content/drive/MyDrive/Colab Notebooks/CCF/DBC_code/src")
!pip install bert4keras

TensorFlow 1.x selected.
     |████████████████████████████████| 51kB 5.1MB/s 
  Created wheel for bert4keras: filename=bert4keras-0.9.6-cp36-none-any.whl size=42954 sha256=5ddc055c59d99d85d660ba7a5e2c4fbcbe83376313c0132e0e4cfb0080568ad3
  Stored in directory: /root/.cache/pip/wheels/b0/95/21/39b1a2052a034a39c63faf81821977d8b64df47961bbf95d52
Successfully built bert4keras


In [3]:
import tensorflow as tf
import bert4keras
import keras
import os
print(tf.__version__)
print(keras.__version__)
print(bert4keras.__version__)
import numpy as np
from bert4keras.backend import keras, K
from bert4keras.models import build_transformer_model
from bert4keras.tokenizers import Tokenizer
from bert4keras.optimizers import Adam
from bert4keras.snippets import sequence_padding, DataGenerator
from bert4keras.snippets import open, ViterbiDecoder, to_array
from bert4keras.layers import ConditionalRandomField
from keras.layers import Dense
from keras.models import Model
from tqdm import tqdm
from  utils import utils
from imp import reload
reload(utils)

Using TensorFlow backend.


1.15.2
2.3.1
0.9.6


<module 'utils.utils' from '/content/drive/MyDrive/Colab Notebooks/CCF/DBC_code/src/utils/utils.py'>

In [4]:
dict_path = '../model/chinese_L-12_H-768_A-12/vocab.txt'
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)
str_="18618193311"
print(tokenizer.encode(str_)[0][1:-1])#[9833, 8662, 8818, 8921, 8452]
tokens = tokenizer.tokenize(str_)#['[CLS]', '186', '##18', '##19', '##33', '##11', '[SEP]']
print("tokens",tokens)
mapping = tokenizer.rematch(str_, tokens)#[[], [0, 1, 2], [3, 4], [5, 6], [7, 8], [9, 10], []]
print("mapping",mapping)
token_ids = tokenizer.tokens_to_ids(tokens)#[101, 9833, 8662, 8818, 8921, 8452, 102]
print("token_ids",token_ids)

[9833, 8662, 8818, 8921, 8452]
tokens ['[CLS]', '186', '##18', '##19', '##33', '##11', '[SEP]']
mapping [[], [0, 1, 2], [3, 4], [5, 6], [7, 8], [9, 10], []]
token_ids [101, 9833, 8662, 8818, 8921, 8452, 102]


In [5]:
train_data = utils.load_data('../data/train_aug.txt')
valid_data = utils.load_data('../data/val_aug.txt')

》	I-movie
[]


In [6]:
ret_train_lst = []
for data in train_data:
    ret = 0
    for char,_ in data:
        ret += len(char)
    ret_train_lst.append(ret)
ret_val_lst = []
for data in valid_data:
    ret = 0
    for char,_ in data:
        ret += len(char)
    ret_val_lst.append(ret)

In [7]:
print(max(ret_train_lst),min(ret_train_lst))
print(max(ret_val_lst),min(ret_val_lst))
print(len(ret_train_lst))
print(len(ret_val_lst))

344 0
300 5
48418
1352


In [8]:
# 预训练模型的超参数
maxlen = 300
epochs = 2 #15
batch_size = 8
bert_layers = 12
learing_rate = 1e-5 
crf_lr_multiplier = 1000 
rnn_lr_multiplier = 1000
# bert配置
config_path = '../model/chinese_L-12_H-768_A-12/bert_config.json'
checkpoint_path = '../model/chinese_L-12_H-768_A-12/bert_model.ckpt'
dict_path = '../model/chinese_L-12_H-768_A-12/vocab.txt'
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)

# 类别映射
labels = [
    'position',
    'name',
    'organization',
    'company',
    'address',
    'movie',
    'game',
    'government',
    'scene',
    'book',
    'mobile',
    'email',
    'QQ',
    'vx',
]

# 0 表示 'O'
# 其他数字表示对应的 B 和 I
# 其他数字表示对应的 B,I,E,S 
id2label = dict(enumerate(labels))
label2id = {j: i for i, j in id2label.items()}
#num_labels = len(labels) * 4 + 1  #BIESO
num_labels = len(labels) * 2 + 1

In [9]:
model = build_transformer_model(
    config_path,
    checkpoint_path,
)

output_layer = 'Transformer-%s-FeedForward-Norm' % (bert_layers - 1)
output = model.get_layer(output_layer).output
output = Dense(num_labels)(output)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [10]:
CRF = ConditionalRandomField(lr_multiplier=crf_lr_multiplier)
output = CRF(output)
model = Model(model.input, output)
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Input-Token (InputLayer)        (None, None)         0                                            
__________________________________________________________________________________________________
Input-Segment (InputLayer)      (None, None)         0                                            
__________________________________________________________________________________________________
Embedding-Token (Embedding)     (None, None, 768)    16226304    Input-Token[0][0]                
__________________________________________________________________________________________________
Embedding-Segment (Embedding)   (None, None, 768)    1536        Input-Segment[0][0]              
____________________________________________________________________________________________

In [11]:
model.compile(
    loss=CRF.sparse_loss,
    optimizer=Adam(learing_rate),
    metrics=[CRF.sparse_accuracy]
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


## 对抗训练

In [12]:
def search_layers(inputs,name,exclude=None):
  """根据inputs和name来搜索层
  说明：inputs为某个层或某个层的输出;name为目标层的名字。
  实现：根据inputs一直往上搜索，直到发现名字为name的层为止
  如果找不到，那就返回None
  """
  if exclude is None:
    exclude=set()
  if isinstance(inputs,keras.layers.Layer):
    layer=inputs
  else:
    layer=inputs._keras_history[0]
  
  if layer.name==name:
    return layer
  elif layer in exclude:#之前搜索过不存在返回None
    return None
  else:
    #向上搜索
    exclude.add(layer)
    inbound_layers=layer._inbound_nodes[0].inbound_layers
    if not isinstance(inbound_layers,list):#inbound_layers 不是list
      inbound_layers=[inbound_layers]#让inbound_layers成list
    if len(inbound_layers)>0:
      for layer in inbound_layers:
        layer=search_layers(layer,name,exclude)#向上搜索
        if layer is not None:#如果不是空则返回
          return layer

def adversarial_training(model,embedding_name,epsilon=1):
  """给模型添加对抗训练
  其中model是需要添加对抗训练的keras模型，embedding_name
  则是model里边Embedding层的名字。要在模型compile之后使用
  """
  if model.train_function is None:
  #compile函数中预定义了一系列训练模型的操作，训练是会调用train_function函数
    model._make_train_function()#手动make
  old_train_function =model.train_function#备份旧的训练函数

  #查找Embedding层
  for output in model.outputs:
    embedding_layer=search_layers(output,embedding_name)
    if embedding_layer is not None:
      break
  if embedding_layer is None:
    raise Exception("Embedding layer not found")
  
  #求embedding梯度
  embeddings=embedding_layer.embeddings#Embedding矩阵
  gradients=K.gradients(model.total_loss,[embeddings])#Embedding梯度
  gradients=K.zeros_like(embeddings)+gradients[0]#转为dense tensor

  #封装为函数
  inputs=(model._feed_inputs+model._feed_targets+model._feed_sample_weights)#所有输入层
  embedding_gradients=K.function(
      inputs=inputs,
      outputs=[gradients],
      name="embedding_gradients"
  )
  def train_function(inputs):#重新定义训练函数
    grades=embedding_gradients(inputs)[0]#Embedding梯度
    delta=epsilon*grades/(np.sqrt((grades**2).sum())+1e-8)#计算扰动
    K.set_value(embeddings,K.eval(embeddings)+delta)#注入扰动
    outputs=old_train_function(inputs)#梯度下降 这里是训练模型更新参数的意思
    K.set_value(embeddings,K.eval(embeddings)-delta)#删除扰动扰动
    return outputs

  model.train_function=train_function


In [13]:
adversarial_train=False
if adversarial_train:
  adversarial_training(model,"Embedding-Token")

In [ ]:
class data_generator(DataGenerator):
    """
    数据生成器
    """
    def __iter__(self, random=False):
        batch_token_ids, batch_segment_ids, batch_labels = [], [], []
        # is_end  是否到 epoch 的末尾， item 对应的 train_data[i]
        for is_end, item in self.sample(random):
            # vocab.txt 从 0 开始计数，0:[pad] 1-99:[unused*] 100:[UNK] [CLS]:101
            # token_ids:101
            token_ids, labels = [tokenizer._token_start_id], [0]
            for w, l in item:
                # 得到除了 [CLS] [SEP] 之外的每一个词的 ids
                # e.g '全国青联委员经纪人' -> [1059, 1744, 7471, 5468, 1999, 1447, 5307, 5279, 782]
                w_token_ids = tokenizer.encode(w)[0][1:-1]
                if len(token_ids) + len(w_token_ids) < maxlen:
                    # e.g [101, 1059, 1744, 7471, 5468, 1999, 1447, 5307, 5279, 782]
                    # 把 [CLS] 加在前面
                    token_ids += w_token_ids
                    if l == 'O':
                        # 如果是 'O' 则标签是 0
                        labels += [0] * len(w_token_ids)
                    else:
                        # 如果不是 'O' 生成 label2id 对应的标签
                        # e.g '全国青联委员经纪人' -> [0, 1, 2, 2, 2, 2, 2, 2, 2, 2] 注意：第一个 [CLS] 默认是 0
                        B = label2id[l] * 2 + 1
                        I = label2id[l] * 2 + 2
                        labels += ([B] + [I] * (len(w_token_ids) - 1))
                    # else:
                    #   #BIESO
                    #   token_len=len(w_token_ids)
                    #   if token_len==1:
                    #     S=label2id[l]*4+4
                    #     label+=([S])
                    #   elif token_len==2:
                    #     B=label2id[l]*4+1
                    #     E=label2id[l]*4+3
                    #     label+=([B]+[E])
                    #   else:
                    #     B=label2id[l]*4+1
                    #     I=label2id[l]*4+2
                    #     E=label2id[l]*4+3
                    #     label+=([B]+[I]*(token_len-2)+[E])
                else:
                    # 超过 maxlen 长度的部分被舍弃掉
                    break
            # 加上尾巴 [SEP]:102
            token_ids += [tokenizer._token_end_id]
            # 尾巴的 label 也是 0
            labels += [0]
            # 生成 segment_ids ，NER 只用了一个句子，所以这里都是 0
            segment_ids = [0] * len(token_ids)
            # 把样本组装成 batch
            batch_token_ids.append(token_ids)
            batch_segment_ids.append(segment_ids)
            batch_labels.append(labels)
            # 如果凑齐了 batchsize 个，或者到 epoch 的最后一个，那么就返回。
            if len(batch_token_ids) == self.batch_size or is_end:
                # 进行 padding 操作
                # [bs,seq] 默认按照 bs 中最大的长度进行填充，保证每个 bs 的长度是一致，用 0 填充
                batch_token_ids = sequence_padding(batch_token_ids)
                batch_segment_ids = sequence_padding(batch_segment_ids)
                batch_labels = sequence_padding(batch_labels)
                yield [batch_token_ids, batch_segment_ids], batch_labels
                batch_token_ids, batch_segment_ids, batch_labels = [], [], []



class NamedEntityRecognizer(ViterbiDecoder):
    """
    命名实体识别器
    """
    def recognize(self, text):
        tokens = tokenizer.tokenize(text)#['[CLS]', '186', '##18', '##19', '##33', '##11', '[SEP]']
        mapping = tokenizer.rematch(text, tokens)#[[], [0, 1, 2], [3, 4], [5, 6], [7, 8], [9, 10], []]
        token_ids = tokenizer.tokens_to_ids(tokens)#[101, 9833, 8662, 8818, 8921, 8452, 102]
        segment_ids = [0] * len(token_ids)
        token_ids, segment_ids = to_array([token_ids], [segment_ids])
        nodes = model.predict([token_ids, segment_ids])[0]
        labels = self.decode(nodes)
        entities, starting = [], False
        for i, label in enumerate(labels):
            if label > 0:
                if label % 2 == 1:
                    starting = True
                    entities.append([[i], id2label[(label - 1) // 2]])
                elif starting:
                    entities[-1][0].append(i)
                else:
                    starting = False
            else:
                starting = False
        # for i,label in enumerate(labels):
        # BIES
        #   if label >0:
        #     if label%4==1:# B-实体标签 标签
        #       starting=True
        #       entities.append([[i],id2label[(label-1)//4]])# 0是CLS和SEQ的标签
        #     elif label%4==0:#S-实体标签 标签 
        #       starting=False
        #       entities.append([[i],id2label[(label-4)//4]])
        #     elif starting:
        #       entities[-1][0].append(i)
        #     else:
        #       starting=False
        #   else:
        #     starting=False

        return [(text[mapping[w[0]][0]:mapping[w[-1]][-1] + 1], l)
                for w, l in entities]


NER = NamedEntityRecognizer(trans=K.eval(CRF.trans), starts=[0], ends=[0])


def evaluate(data):
    """评测函数
    """
    X, Y, Z = 1e-10, 1e-10, 1e-10
    for d in tqdm(data):
        # d: [[text,label],...]
        text = ''.join([i[0] for i in d])
        # 预测标签 {('卡卡','name'),('深度之眼','organization'),...}
        R = set(NER.recognize(text)) 
        # 真实标签 {('卡卡','name'),('深度之眼','organization'),...}
        T = set([tuple(i) for i in d if i[1] != 'O'])
        # 只有标签类别和识别名字完全对应上才会有值
        X += len(R & T) 
        # 所有预测为 “正” 样本的标签
        Y += len(R) 
        # 所有真实为 “正” 样本的标签
        Z += len(T)
    # 查准率和查全率
    precision, recall =  X / Y, X / Z
    f1 = 2*precision*recall/(precision+recall)
    return f1, precision, recall


class Evaluator(keras.callbacks.Callback):
    def __init__(self,valid_data):
        self.best_val_f1 = 0
        self.valid_data = valid_data

    def on_epoch_end(self, epoch, logs=None):
        trans = K.eval(CRF.trans)
        NER.trans = trans
#         print(NER.trans)
        # 计算验证集的评估指标
        f1, precision, recall = evaluate(self.valid_data)
        # 保存最优
        if f1 >= self.best_val_f1:
            self.best_val_f1 = f1
            model.save_weights('../model/best_model_epoch_10.weights')
        print(
            'valid:  f1: %.5f, precision: %.5f, recall: %.5f, best f1: %.5f\n' %
            (f1, precision, recall, self.best_val_f1)
        )



evaluator = Evaluator(valid_data)
train_generator = data_generator(train_data, batch_size)

model.fit_generator(
    train_generator.forfit(),
    steps_per_epoch=len(train_generator),
    epochs=epochs,
    callbacks=[evaluator]
)



Epoch 1/2
2783/6053 [============>.................] - ETA: 19:46 - loss: 7.0730 - sparse_accuracy: 0.8540

In [ ]:
# 数据处理参数
symbol = ['？','⋯','…','﹗']
max_sent_length = 250
max_input_length = 300     
# 建立分词器
tokenizer = Tokenizer(dict_path, do_lower_case=True)
# 类别映射
labels = [
    'position',
    'name',
    'organization',
    'company',
    'address',
    'movie',
    'game',
    'government',
    'scene',
    'book',
    'mobile',
    'email',
    'QQ',
    'vx',
]
# 0 表示 'O'
# 其他数字表示对应的 B 和 I
id2label = dict(enumerate(labels))
label2id = {j: i for i, j in id2label.items()}
num_labels = len(labels) * 2 + 1

In [ ]:
model_path = '../model/best_model_epoch_10.weights'
model.load_weights(model_path)

In [ ]:
class NamedEntityRecognizer(ViterbiDecoder):
    """
    命名实体识别器
    """
    def recognize(self, text):
        tokens = tokenizer.tokenize(text)
        mapping = tokenizer.rematch(text, tokens)
        token_ids = tokenizer.tokens_to_ids(tokens)
        segment_ids = [0] * len(token_ids)
        nodes = model.predict([[token_ids], [segment_ids]])[0]
        labels = self.decode(nodes)
        entities, starting = [], False
        
        for i, label in enumerate(labels):
            if label > 0:
                if label % 2 == 1:
                    starting = True
                    entities.append([[i], id2label[(label - 1) // 2]])
                elif starting:
                    entities[-1][0].append(i)
                else:
                    starting = False
            else:
                starting = False

        return [(text[mapping[w[0]][0]:mapping[w[-1]][-1] + 1], l)
                for w, l in entities]
        #[(中国，ORG)，（12345678912，mobile）,.....]
    
    
NER = NamedEntityRecognizer(trans=K.eval(CRF.trans), starts=[0], ends=[0])

In [ ]:
def test_predict(data, NER_):
    test_ner =[]
    for text in tqdm(data):
        cut_text_list, cut_index_list = utils.agg_sent([text],symbol, max_sent_length, max_input_length)
        posit = 0
        item_ner = []
        index =1
        for str_ in cut_text_list:#[str_1,str_2] str_：一段字符串
            ner_res  = NER_.recognize(str_)
            for tn in ner_res:
                ans = {}
                ans["label_type"] = tn[1]#标签
                ans['index'] = str(index)#一个txt识别预测出来的多少个实体
                ans["start_pos"] = text.find(tn[0],posit)
                ans["end_pos"] = ans["start_pos"] + len(tn[0])-1
                posit = ans["end_pos"]
                ans["res"] = tn[0]#对应的实体字符串
                item_ner.append(ans)#[{},{},{},{}]
                index +=1
        test_ner.append(item_ner)#[[{},{},{},{},{},{},{},{}],[{},{},{},{},{},{},{},{}]]
                       #text1     
    return test_ner

In [ ]:
from glob import glob
test_files = glob("../data/test_data/*.txt")

In [ ]:
df_ret = {'ID':[],'Category':[],'Pos_b':[],'Pos_e':[],'Privacy':[]}
for file in test_files:
    with open(file, "r", encoding="utf-8") as f:
        line = f.read()
        line = [line]
        ret = test_predict(line, NER)
    idx = os.path.basename(file).split('.')[0]
    for line in ret[0]:#这里取【0】的原因是open开的是一段完整的文字，所以识别后返回的也是一段的实体识别标签
        df_ret['ID'].append(idx)
        df_ret['Category'].append(line['label_type'].strip('\n'))
        df_ret['Pos_b'].append(line['start_pos'])
        df_ret['Pos_e'].append(line['end_pos'])
        df_ret['Privacy'].append(line['res'].replace('\n',''))

# Submit 提交


In [ ]:
import pandas as pd
version = '20201101'
df_ret_ = pd.DataFrame(df_ret)
df_ret_ = df_ret_.sort_values('ID')
df_ret_.to_csv('../submit/predict{}.csv'.format(version),index=None)

In [ ]:
file_name = '../submit/predict{}.csv'.format(version)
utils.checkout(file_name)

## 后处理的部分

In [ ]:
import pandas as pd
import os 
os.chdir("/content/drive/MyDrive/Colab Notebooks/CCF/DBC_code/src")
pred=pd.read_csv('../submit/predict20201101.csv')
pred.head()

In [ ]:
import re
reg = re.compile(r'(《.*》)')
#re.findall返回的是list，没有匹配成功就是
uncomplete_yinhao_book = pred[pred.Category == 'book'][(pred[pred.Category == 'book'].Privacy.apply(c:re.findall(reg,x)).apply(lambda x:len(x))!=1)]
temp =    pred[pred.Category == 'book'][(pred[pred.Category == 'book'].Privacy.apply(lambda x:re.findall(reg,x)).apply(lambda x:len(x))!=1)]
half_yinghao_book = temp[temp.Privacy.apply(lambda x :('《' in x or '》' in x))]
uncomplete_yinhao_book.head()

In [ ]:
half_yinghao_book.head()

In [ ]:
from tqdm import tqdm
def repair_book(bad_data_book,max_len = 20):
    ret = {'ID':[],'Category':[],'Pos_b':[],'Pos_e':[],'Privacy':[]}
    for idx,cat,pos_b,pos_e,privacy in tqdm(bad_data_book.values):
        if '《' in privacy:
            with open('../data/test_data/{}.txt'.format(idx),'r') as f:
                content = f.readline()
            for i in range(1, min(len(content)-pos_e-1,max_len)):
                index  = pos_e + i
                if content[index] in (',.。!！《'):
                    break
                if content[index] == '》':
                    pos_e = index
                    break
            ret['ID'].append(idx)
            ret['Category'].append(cat)
            ret['Pos_b'].append(pos_b)
            ret['Pos_e'].append(pos_e)
            ret['Privacy'].append(content[pos_b:pos_e+1])
        if '》' in privacy:
            with open('../data/test_data/{}.txt'.format(idx),'r') as f:
                content = f.readline()
            for i in range(1,min(pos_b,max_len)):
                index  = pos_b -1 - i
                if content[index] in (',.。!！》'):
                    break
                if content[index] == '《':
                    pos_b = index
                    break
            ret['ID'].append(idx)
            ret['Category'].append(cat)
            ret['Pos_b'].append(pos_b)
            ret['Pos_e'].append(pos_e)
            ret['Privacy'].append(content[pos_b:pos_e+1])
    return ret


In [ ]:
repair_ret=repair_book(half_yinghao_book,max_len = 20)
repair_ret=pd.DataFrame(repair_ret)
repair_ret.head()

In [ ]:
pred.loc[half_yinghao_book.index]=repair_ret.values

In [ ]:
pred.loc[half_yinghao_book.index].head()
#GameInformer》 这几个数据都是数据本身有问题无法搜索到补全《的信息

In [ ]:
pred=pred.drop_duplicates()
print(pred[pred.ID==1127])
pred.to_csv('../submit/predict_postprocess.csv',index=None)

## 差分学习率

In [ ]:
import keras.backend as K
class SetLearningRate:
    """层的一个包装，用来设置当前层的学习率
    引用自 keras4bert 作者代码
    """
    def __init__(self, layer, lamb, is_ada=False):
        self.layer = layer
        self.lamb = lamb # 学习率比例
        self.is_ada = is_ada # 是否自适应学习率优化器

    def __call__(self, inputs):
        with K.name_scope(self.layer.name):
            if not self.layer.built:
                input_shape = K.int_shape(inputs)#inputs的形状
                self.layer.build(input_shape)#重建模型层
                self.layer.built = True#训练True
                if self.layer._initial_weights is not None:
                    self.layer.set_weights(self.layer._initial_weights)#初始化层参数权重
        for key in ['kernel', 'bias', 'embeddings', 'depthwise_kernel', 'pointwise_kernel', 'recurrent_kernel', 'gamma', 'beta']:
#                              模型层的一些可学习参数
            if hasattr(self.layer, key):#查看layer是否有key值的属性
                weight = getattr(self.layer, key)#获取值
                if self.is_ada:
                    lamb = self.lamb # 自适应学习率优化器直接保持lamb比例
                else:
                    lamb = self.lamb**0.5 # SGD（包括动量加速），lamb要开平方
                K.set_value(weight, K.eval(weight) / lamb) # 更改初始化
                setattr(self.layer, key, weight * lamb) # 按比例替换
        return self.layer(inputs)

# 使用方法
# 构建层，然后用 SetLearningRate 包在层的外面，rnn_lr_multiplier 为要放大的学习率倍数，如果是自适应的梯度下降法，设置 is_ada = True
brnn = Bidirectional(GRU(128, return_sequences=True),merge_mode='sum')
brnn_mul = SetLearningRate(brnn,lamb = rnn_lr_multiplier,is_ada = True)